In [1]:
import numpy as np
import random as rnd
import matplotlib.pyplot as plt
import math
import copy
import quadprog
from matplotlib import animation
import json
from pprint import pprint
plt.rc('animation', html='html5')

#### Initialization

In [24]:
data = json.load(open('P21.json'))
dt = 0.01 #data["vehicle_dt"]
agents = []
for i in range(len(data["start_positions"])):
    newEntry = { "position": np.array(data["start_positions"][i]),
        "p_goal": np.array(data["goal_positions"][i]),
        "vel": np.array([0.0, 0.0]),
        "radius": 0.5,
        "v_max": data["vehicle_v_max"]
    }
    #newEntry["position"] += np.random.normal(loc=0.0, scale=0.001, size=2)
    agents.append(newEntry)
tau = 0.01

#### Animation

In [25]:
fig, ax = plt.subplots(figsize=(5,5))
ax.axes.set_xlim((5, 35))
ax.axes.set_ylim((10, 40))

circles = []
for agent in agents:
    newEntry = plt.Circle(agent["position"], agent["radius"], color='blue') #
    ax.add_artist(newEntry)
    circles.append(newEntry)

G = 2.0*np.identity(2)
G_new = 0.000001*np.identity(3)
G_new[2][2] = 1.0

b = np.zeros(len(agents)+7)
C = np.zeros((len(agents)+7,2))
# Adding circular constraints
C[-1] = np.array([-1.0, -1.0])
C[-2] = np.array([-1.0, 1.0])
C[-3] = np.array([1.0, -1.0])
C[-4] = np.array([1.0, 1.0])
b[-1] = -1.4*data["vehicle_v_max"]
b[-2] = -1.4*data["vehicle_v_max"]
b[-3] = -1.4*data["vehicle_v_max"]
b[-4] = -1.4*data["vehicle_v_max"]
C[-5] = np.array([-1.0, 0.0])
C[-6] = np.array([1.0, 0.0])
C[-7] = np.array([0.0, -1.0])
C[-8] = np.array([0.0, 1.0])
b[-5] = -0.98*data["vehicle_v_max"]
b[-6] = -0.98*data["vehicle_v_max"]
b[-7] = -0.98*data["vehicle_v_max"]
b[-8] = -0.98*data["vehicle_v_max"]

totalTime = { "val": 0.0 }

order = np.arange(len(agents))
#order = np.full(len(agents), 1)
#for i in range(len(order)):
#    order[i] = len(order)-i-1

#order = np.full(len(agents), 1)
#order1 = np.full(int(np.ceil(len(agents)/2.0)), 1)
#order2 = np.full(len(agents)-len(order1), 1)
#counter1 = 0
#counter2 = 0
#for i in range(len(agents)):
#    if (i%2 == 0):
#        order1[counter1] = i
#        counter1 += 1
#    else:
#        order2[counter2] = i
#        counter2 += 1
#np.random.shuffle(order1)
#np.random.shuffle(order2)
#order[:len(order1)] = order1
#order[len(order1):] = order2


fin_flag = {"val":False}

def animate(i):
    
    np.random.shuffle(order)
    #np.random.shuffle(order2)
    #order[:len(order1)] = order1
    #order[len(order1):] = order2
    
    finished = True
    for i in order:
        agent = agents[i]
        if (dist(agent["position"], agent["p_goal"]) > agent["v_max"]*dt): #or length(agent["vel"]) > 0.0001):
            finished = False
        if (dist(agent["position"], agent["p_goal"]) < 0.001):
            continue
        
        # Render
        newPos = agent["position"] + dt*agent["vel"]
        collide = False
        for j in range(len(agents)):
            if (j == i):
                continue
            distance = dist(newPos, agents[j]["position"])
            if (distance < (agent["radius"]+agents[j]["radius"])):
                collide = True
                break
        if (not collide):
            agent["position"] = newPos
        #else:
        #    agent["vel"] = np.array([0.0, 0.0])
        circles[i].center = agent["position"]
        
        #if (dist(agent["position"], agent["p_goal"]) < 0.001): #or length(agent["vel"]) > 0.0001):
        #    continue
    
    if (finished):
        if (not fin_flag["val"]):
            print("Finished: " + str(totalTime["val"]))
        fin_flag["val"] = True
        return circles
    
    for i in order:
        agent = agents[i]
    
        # Update velocity
        v_pref = agent["p_goal"]-agent["position"]
        if (dist(agent["position"],agent["p_goal"]) <= agent["v_max"]*dt):
            v_pref /= dt
        else:
            v_pref *= (agent["v_max"]/length(v_pref))
        a = 2.0*v_pref
        
        counter = 0
        for j in range(len(agents)):
            if (j == i):
                continue
            goalReached = False
            if (dist(agents[j]["position"], agents[j]["p_goal"]) <= 0.001 and length(agents[j]["vel"]) <= 0.001):
                goalReached = True
            
            #if (goalReached):
            #    u,n = velObst(agent, agents[j], 0.1)
            #else:
            u,n = velObst(agent, agents[j], tau)
            C[counter] = n
            if (goalReached):
                b[counter] = np.sum(n*(agent["vel"] + u))
            else:
                b[counter] = np.sum(n*(agent["vel"] + 0.5*u))
            counter += 1
        
        try:
            newVel = quadprog.solve_qp(G, a, C.T, b)[0]
        except ValueError:
            # Solve another linear program
            C_new = np.zeros((C.shape[0], C.shape[1]+1))
            C_new[:,:-1] = C
            for j in range(C.shape[0]-8):
                C_new[j][2] = 1.0
            newVel = quadprog.solve_qp(G_new, np.zeros(3), C_new.T, b)[0][:-1]
            
        agent["vel"] = newVel
        #agent["position"] += dt*agent["vel"]
        
    #for i in order:
    #    agents[i]["position"] += dt*agents[i]["vel"]
    
    totalTime["val"] += dt
    
    return circles

ani = animation.FuncAnimation(fig, animate, np.arange(0, 3700), interval=(dt*500), blit=True)
ani.save('video10.mp4')

C:\Users\d1mas\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in true_divide


Finished: 33.96000000000181


In [71]:
37.020000000001204*14/18

28.793333333334267

In [11]:
def velObst(a1, a2, tau):
    
    p1 = a1["position"]
    p2 = a2["position"]
    c0 = (p2-p1)/tau
    r0 = (a1["radius"]+a2["radius"])/tau
    
    c = length(c0)
    alpha = np.arcsin(r0/c)
    c_angle = np.arctan2(c0[1], c0[0])
    dist_ = np.sqrt(c*c-r0*r0)
    
    k1 = np.tan(c_angle+alpha)
    k2 = np.tan(c_angle-alpha)
    
    v_opt = a1["vel"]-a2["vel"]
    
    angle1 = c_angle+alpha
    angle1 = np.arctan2(np.sin(angle1), np.cos(angle1))
    angle2 = c_angle-alpha
    angle2 = np.arctan2(np.sin(angle2), np.cos(angle2))
    
    d1 = float("inf")
    x1 = 0
    y1 = 0
    proj_l = v_opt[0]*np.cos(angle1) + v_opt[1]*np.sin(angle1)
    if (proj_l > 0 and length(v_opt) > dist_):
        x1 = proj_l*np.cos(angle1)
        y1 = k1*x1
        d1 = dist(v_opt,(x1,y1))
    
    d2 = float("inf")
    x2 = 0
    y2 = 0
    proj_l = v_opt[0]*np.cos(angle2) + v_opt[1]*np.sin(angle2)
    if (proj_l > 0 and length(v_opt) > dist_):
        x2 = proj_l*np.cos(angle2)
        y2 = k2*x2
        d2 = dist(v_opt,(x2,y2))
    
    # Tangents to the circle
    k = (c0[1]-v_opt[1])/(c0[0]-v_opt[0])
    b = v_opt[1] - v_opt[0]*k
    a = 1+k*k
    eq_b = -2*c0[0]+2*k*b-2*k*c0[1]
    c = c0[0]*c0[0] + b*b + c0[1]*c0[1] - 2*b*c0[1] - r0*r0
    
    d_sqrt = np.sqrt(eq_b*eq_b - 4*a*c)
    x_sol1 = (-eq_b-d_sqrt)/(2*a)
    x_sol2 = (-eq_b+d_sqrt)/(2*a)
    y_sol1 = k*x_sol1+b
    y_sol2 = k*x_sol2+b
    
    d5 = float("inf")
    x5 = 0
    y5 = 0
    if (length((x_sol1,y_sol1)) < dist_):
        x5 = x_sol1
        y5 = y_sol1
        d5 = dist(v_opt,(x5,y5))
        
    d6 = float("inf")
    x6 = 0
    y6 = 0
    if (length((x_sol2,y_sol2)) < dist_):
        x6 = x_sol2
        y6 = y_sol2
        d6 = dist(v_opt,(x6,y6))
    
    # Find closest point
    u = None
    if (d1 <= d2 and d1 <= d5 and d1 <= d6):
        u = np.array([x1,y1]) - v_opt
    elif (d2 <= d1 and d2 <= d5 and d2 <= d6):
        u = np.array([x2,y2]) - v_opt
    elif (d5 <= d1 and d5 <= d2 and d5 <= d6):
        u = np.array([x5,y5]) - v_opt
    else:
        u = np.array([x6,y6]) - v_opt
    
    opt_in_obs = False
    opt_angle = np.arctan2(v_opt[1],v_opt[0])
    if (opt_angle > np.min((angle1, angle2)) and opt_angle < np.max((angle1, angle2))):
        if (length(v_opt) > dist_ or dist(v_opt, c0) < r0):
            opt_in_obs = True
    
    n = copy.deepcopy(u)
    if (not opt_in_obs):
        n = -n
    n /= length(n)
    
    return u, n

In [10]:
def dist(p1,p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
def length(p):
    return np.sqrt(p[0]*p[0] + p[1]*p[1])

#### Debugging

In [55]:
### TEST
tau = 1.0
agent1 = { "position": np.array([0.0,0.0]),
          "vel": np.array([4.5,1.5]),
          "radius": 0.5
    }
agent2 = { "position": np.array([2.0,2.0]),
          "vel": np.array([0.0,0.0]),
          "radius": 0.5
    }